In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


In [2]:
using JuMP
using HiGHS

# Goal Programming Approach

- I am 3 digit number between 400 and 800
- My digits add up to 15
- My ten's digit is twice hundereds' digit
- What number am I?

$$
\begin{aligned}
\min z = & d_1^- + d_1^+ \\
\text{Subject to:} \\
& x_1 + x_2 + x_3 + d_1^- - d_1^+ = 15 \\
& x_2 - 2x_1 = 0 \\
& 100x_1 + 10x_2 + x_3 \le 800 \\
& 100x_1 + 10x_2 + x_3 \ge 400 \\
& x_1, x_2, x_3 \ge 0 \\
& x_1, x_2, x_3 \le 9 \\
& d_1^-, d_1^+ \ge 0 \\
& x_1, x_2, x_3 \in \{0, Z^+\}
\end{aligned}
$$

In [3]:
m = Model(HiGHS.Optimizer)

@variable(m, x[1:3], Int)
@variable(m, d[1:2])

@objective(m, Min, sum(d[1:2]))
@constraint(m, x[1] + x[2] + x[3] + d[1] - d[2] == 15)
@constraint(m, x[2] == 2 * x[1])
@constraint(m, x[1] * 100 + x[2] *10 + x[3] <= 800)
@constraint(m, x[1] * 100 + x[2] *10 + x[3] >= 400)
@constraint(m, x[1:3] .>= 0)
@constraint(m, x[1:3] .<= 9)
@constraint(m, d[1:2] .>= 0)

optimize!(m)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 12 rows; 5 cols; 21 nonzeros; 3 integer variables (0 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+02]
  Cost    [1e+00, 1e+00]
  Bound   [0e+00, 0e+00]
  RHS     [9e+00, 8e+02]
Presolving model
3 rows, 4 cols, 8 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
Presolve reductions: rows 1(-11); columns 3(-2); nonzeros 3(-18) 

Solving MIP model with:
   1 row
   3 cols (0 binary, 1 integer, 0 implied int., 2 continuous, 0 domain fixed)
   3 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds 

In [4]:
value.(x[1:3])

3-element Vector{Float64}:
 4.0
 8.0
 3.0

In [7]:
JuMP.latex_formulation(m)

$$ \begin{aligned}
\text{feasibility}\\
\text{Subject to} \quad & a + b + c = 15\\
 & -2 a + b = 0\\
 & 100 a + 10 b + c \in [400, 800]\\
 & a \in [0, 9]\\
 & b \in [0, 9]\\
 & c \in [0, 9]\\
 & a \in \mathbb{Z}\\
 & b \in \mathbb{Z}\\
 & c \in \mathbb{Z}\\
\end{aligned} $$

# Constraint Satisfaction Approach

- I am 3 digit number between 400 and 800
- My digits add up to 15
- My ten's digit is twice hundereds' digit
- What number am I?

In [6]:
m = Model(HiGHS.Optimizer)
@variable(m, a, Int)
@variable(m, b, Int)
@variable(m, c, Int)

@constraint(m, 400 <= 100a + 10b + c <= 800)
@constraint(m, a + b + c == 15)
@constraint(m, b == 2a)
@constraint(m, 0 .<= [a, b, c] .<= 9)

optimize!(m)
value.([a, b, c])

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 6 rows; 3 cols; 11 nonzeros; 3 integer variables (0 binary)
Coefficient ranges:
  Matrix  [1e+00, 1e+02]
  Cost    [0e+00, 0e+00]
  Bound   [0e+00, 0e+00]
  RHS     [9e+00, 8e+02]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-6); columns 0(-3); nonzeros 0(-11) - Reduced to empty
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. 

3-element Vector{Float64}:
 4.0
 8.0
 3.0

In [8]:
JuMP.latex_formulation(m)

$$ \begin{aligned}
\text{feasibility}\\
\text{Subject to} \quad & a + b + c = 15\\
 & -2 a + b = 0\\
 & 100 a + 10 b + c \in [400, 800]\\
 & a \in [0, 9]\\
 & b \in [0, 9]\\
 & c \in [0, 9]\\
 & a \in \mathbb{Z}\\
 & b \in \mathbb{Z}\\
 & c \in \mathbb{Z}\\
\end{aligned} $$